In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [ ]:
def load_heat_gradients(csv_path):
    df = pd.read_csv(csv_path)  
    
    anchors = df.values[:, :3]
    hgs_fem = df.values[:, 3:6]
    hgs_dec = df.values[:, 6:9]
    
    hgs_reg = None
    if df.shape[1] == 12:
        hgs_reg = df.values[:, 9:12]
    
    anchor_norms = np.linalg.norm(anchors, axis=1)
    
    # filter out origin if it exists
    norigin_mask = anchor_norms != 0
    
    anchors = anchors[norigin_mask]
    hgs_fem = hgs_fem[norigin_mask]
    hgs_dec = hgs_dec[norigin_mask]
    if hgs_reg is not None:
        hgs_reg = hgs_reg[norigin_mask]
    
    anchor_norms = anchor_norms[norigin_mask]
    anchors_normed = anchors / anchor_norms[:,None]
    
    hgs_fem_normed = - hgs_fem / np.linalg.norm(hgs_fem, axis=1, keepdims=True)
    hgs_dec_normed = - hgs_dec / np.linalg.norm(hgs_dec, axis=1, keepdims=True)
    if hgs_reg is not None:
        hgs_reg_normed = -hgs_reg / np.linalg.norm(hgs_reg, axis=1, keepdims=True)
        
    dot_products_fem = np.einsum('ij,ij->i', anchors_normed, hgs_fem_normed)
    dot_products_dec = np.einsum('ij,ij->i', anchors_normed, hgs_dec_normed)
    
    if hgs_reg is not None:
        dot_product_reg = np.einsum('ij,ij->i', anchors_normed, hgs_reg_normed)
    
    names = ['fem', 'dec']
    products = [dot_products_fem, dot_products_dec]
    if hgs_reg is not None:
        names.append('reg')
        products.append(dot_product_reg)
    
    return names, anchors, products

def compare_heat_gradients_cv(csv_prefix):
    
    metric_functions = {
        'mean'                 : lambda x : x.mean(),
        'mean_topdiff'         : lambda x : (1 - x).mean(),
        'max_topdiff'          : lambda x : (1 - x).max(),
        'mean_topdiff_squared' : lambda x : ((1 - x) ** 2).mean()
    }
    
    print('CSV PREFIX: {}'.format(csv_prefix))
    
    csvp_cells = csv_prefix + '_cells.csv'
    csvp_verts = csv_prefix + '_vertices.csv'
    
    names, anchors_cells, products_cells = load_heat_gradients(csvp_cells)
    names, anchors_verts, products_verts = load_heat_gradients(csvp_verts)
    
    # join metrics in a df
    value_dict = {'anchor'    : ['cell'] * len(products_cells) + ['vertex'] * len(products_verts), 
                  'laplacian' : names * 2 }
    for m_k, m in metric_functions.items():
        value_dict[m_k] = [m(pc) for pc in products_cells] + [m(pv) for pv in products_verts]
                  
    reg_included = False
    if (len(products_cells) == 3):
        reg_included = True

    df = pd.DataFrame(value_dict)
    display(df.set_index(['anchor', 'laplacian']))
    
    anchor_norms_cells = np.linalg.norm(anchors_cells, axis=1)
    anchor_norms_verts = np.linalg.norm(anchors_verts, axis=1)
    
    # PLOTS
    rows = 2
    cols = 2 if not reg_included else 3
    fig, axs = plt.subplots(rows, cols, figsize=(10, 5))
    
    # cell-based FEM
    axs[0, 0].set_title('Cells, {}'.format(names[0]))
    axs[0, 0].scatter(anchor_norms_cells, products_cells[0], s=0.5)
    axs[0, 0].set_ylim([-1, 1])
    axs[0, 1].set_title('Cells, {}'.format(names[1]))
    axs[0, 1].scatter(anchor_norms_cells, products_cells[1], s=0.5)
    axs[0, 1].set_ylim([-1, 1])
    axs[1, 0].set_title('Verts, {}'.format(names[0]))
    axs[1, 0].scatter(anchor_norms_verts, products_verts[0], s=0.5)
    axs[1, 0].set_ylim([-1, 1])
    axs[1, 1].set_title('Verts, {}'.format(names[1]))
    axs[1, 1].scatter(anchor_norms_verts, products_verts[1], s=0.5)
    axs[1, 1].set_ylim([-1, 1])
    
    if (reg_included):
        axs[0, 2].set_title('Cells, {}'.format(names[2]))
        axs[0, 2].scatter(anchor_norms_cells, products_cells[2], s=0.5)
        axs[0, 2].set_ylim([-1, 1])
        axs[1, 2].set_title('Verts, {}'.format(names[2]))
        axs[1, 2].scatter(anchor_norms_verts, products_verts[2], s=0.5)     
        axs[1, 2].set_ylim([-1, 1])
    
    plt.tight_layout()
    plt.show()
    fig.clf()
    
    return df

def plot_ifo_val(df, by_val):
    clrs = ['r', 'b', 'g']
    lss  = ['-', '--']
    
    index_cols = ['anchor', 'laplacian', by_val]
    metrics = [m for m in df.keys() if m not in index_cols]
    
    for m in metrics:
        ax = plt.gca()
        for a_idx, an in enumerate(df['anchor'].unique()): 
            for l_idx, ln in enumerate(df['laplacian'].unique()):
                df[(df['anchor'] == an) & (df['laplacian'] == ln)].plot(x=by_val, y=m, label='{} {}'.format(an, ln), ax=ax, c=clrs[l_idx], ls=lss[a_idx], alpha=0.8)
        plt.title(m)
        plt.show()

# Stats IFO timestep t

In [ ]:
meandfs = []
expsteps = [-4, -3, -2, -1, 0]
cellSize = 0.1
for texp in expsteps:
    foldername = 'KA_hd_t1e{}'.format(texp)
    folderpath = '../build/out/{}/plots'.format(foldername)
    meshestxt = folderpath + '/meshes.txt'
    meshesprefix = folderpath + '/OriginSphere{}_-1.000000_R0_heatgradients'.format(cellSize)
    meandfs.append(compare_heat_gradients_cv(meshesprefix))

# add the time labels
for e_idx, es in enumerate(expsteps):
    meandfs[e_idx]['t'] = es
meandf = pd.concat(meandfs)

In [ ]:
plot_ifo_val(meandf, 't')

# Regnoise

In [ ]:
meandfs_noise = []
cellSize = 0.1
noises = [-2, 0., 0.1, 0.2, 0.5, 1.0, 2.]
for noisescale in noises:
    foldername = 'KC_hd_t1e-3_noisenreg'.format(texp)
    folderpath = '../build/out/{}/plots'.format(foldername)
    meshestxt = folderpath + '/meshes.txt'
    meshesprefix = folderpath + '/OriginSphere{}_{:6f}_R0_heatgradients'.format(cellSize, noisescale)
    meandfs_noise.append(compare_heat_gradients_cv(meshesprefix))
    
# add the noise labels
for n_idx, noisescale in enumerate(noises):
    meandfs_noise[n_idx]['regnoise'] = noisescale
meandf_noise = pd.concat(meandfs_noise)

In [ ]:
plot_ifo_val(meandf_noise, 'regnoise')